    ✍ Теперь, когда вы знаете основные классы задач машинного обучения, настало время перейти к предварительной обработке (препроцессингу, preprocessing) данных. Препроцессинг в машинном обучении — это важный шаг, который помогает повысить качество данных с целью извлечения из них значимых идей.

    Как правило, реальные данные являются неполными, непоследовательными, неточными (содержат ошибки или выбросы). Препроцессинг данных помогает очищать, форматировать и упорядочивать необработанные данные, тем самым делая их готовыми к работе с моделями машинного обучения.

**В этом модуле мы повторим:**

* какие способы представления данных существуют;
* почему к разным представлениям данных нужны разные подходы;
* как происходит кодирование признаков;
* какие способы заполнения пропусков в данных существуют.

Мы разберём:| Вы научитесь:
---------|----------
что такое разбиение по бинам и как его осуществить;| ыбирать подходящий метод кодирования признаков в зависимости  от типа модели машинного обучения;
что такое трансформация распределения и какие её разновидности существуют;| подготавливать данные к обучению;
что такое масштабирование признаков;| производить анализ влияния отбора и селекции признаков на качество модели с помощью Comet ML.
как работать с датами и расстояниями;| 
какие алгоритмы машинного обучения более чувствительны к предобработке данных.| 

Обратимся к нашим данным. Нас просят предсказать цену автомобиля. Для обучения модели выделим целевой столбец:

In [1]:
import pandas as pd
from sklearn.linear_model import LinearRegression

In [2]:
data = pd.read_excel('data/data_ford_price.xlsx') 

In [3]:
y = data['price']
X = data.drop(columns='price')

In [4]:
lr = LinearRegression()
lr.fit(X,y)

ValueError: could not convert string to float: 'clean'

Мы получим ошибку с комментарием о том, что не удалось превратить строковое значение в число с плавающей точкой (float).

Чтобы этой ошибки не возникало, необходимо закодировать данные такими методами кодирования, как:

* порядковое кодирование;
* двоичное кодирование;
* однократное кодирование.

![](./image/asset-v1_SkillFactory+DST-3.0+28FEB2021+type@asset+block@dst3-ml6-3_2.png)

В таблице ниже представлено сравнение данных способов кодировки:

Значения признака «Образование»| Порядковое кодирование| Двоичное кодирование| Однократное кодирование
------|-------|-------|--------
Нет| 1| 000| 000001
Начальное| 2| 001| 000010
Среднее| 3| 010| 000100
BSc| 4| 011| 001000
MSc| 5| 100| 010000
PhD| 6| 101| 100000

Для реализации данных методов мы использовали библиотеку `category_encoders`. Однако вы уже знаете и такой мощный инструмент, как *scikit-learn*. Данная библиотека содержит набор реализованных алгоритмов машинного обучения, метрик для оценки их качества, а также  класс `preprocessing` для предобработки данных, в частности — для кодирования категориальных признаков.

Представленная ниже таблица показывает соответствие типа кодирования классу в `sklearn.processing`.

Тип кодировки| Класс библиотеки sklearn
------|--------
Порядковое кодирование| LabelEncoder
Двоичное кодирование| LabelBinarizer
Однократное кодирование| OneHotEncoder

![](./image/asset-v1_SkillFactory+DST-3.0+28FEB2021+type@asset+block@dst3-ml6-3_3.png)

>Преимущество использования одной библиотеки состоит в типичности методов. Например, вы знаете, что для обучения модели в sklearn используется метод `fit()`. При кодировании признаков здесь также применяют `fit()` для подгонки кодировщика под выборку и `transform()` — для преобразования данных в числа.

**Алгоритм кодирования в sklearn следующий:**

![](./image/asset-v1_SkillFactory+DST-3.0+28FEB2021+type@asset+block@dst3-ml6-3_4.png)

Из предыдущих модулей мы знаем, что при решении задач машинного обучения данные разбираются на обучающую (*train*) и валидационную (*validation*) выборки (последняя также может быть тестовой (*test*) выборкой). По аналогии подгонка кодировщика происходит на обучающей выборке, а трансформация — на обучающей и на тестовой.

Почему так? Потому что наша обученная модель не должна видеть данные, которые подаются в неё на тесте. Только так мы можем судить о том, что модель обучена качественно. То же самое и с кодировкой.

Давайте посмотрим на кодирование признака Образование способом «один-против-всех» (*one vs all*):

In [5]:
from sklearn.preprocessing  import LabelBinarizer
 
lb = LabelBinarizer()
 
education = ['нет', 'начальное', 'среднее', 'BSc', 'MSc', 'начальное', 'PhD']
 
lb.fit(education)
 
print('категории:', lb.classes_)
 
lb.transform(['нет', 'MSc'])

категории: ['BSc' 'MSc' 'PhD' 'начальное' 'нет' 'среднее']


array([[0, 0, 0, 0, 1, 0],
       [0, 1, 0, 0, 0, 0]])

У класса `LabelBinarizer`, как и у двух остальных, есть атрибут `classes_`, который выводит список уникальных значений признака.

Вернёмся к нашей выборке. В ней присутствуют следующие категориальные признаки: `condition`, `cylinders`, `title_status`, `transmission`, `drive`, `size`.

При этом признаки `condition` и `cylinders` — числовые, а `title_status`, `transmission`, `drive`, `size` — текстовые.

![](./image/asset-v1_SkillFactory+DST-3.0+28FEB2021+type@asset+block@dst3-ml6-3_6.png)

Важно производить кодирование номинальных признаков, даже если они уже представлены в числовом формате, так как, в отличие от порядковых признаков, **категории номинальных являются независимыми**. В случае порядкового кодирования таких признаков мы вносим искусственные закономерности в данные (например, чем больше числовой код цилиндров, тем лучше, хотя это необязательно так).

Посмотрим на число уникальных значений номинальных признаков `title_status`, `transmission`, `drive`, `size` и `cylinders`:

In [6]:
columns_to_change = ['cylinders', 'title_status', 'transmission', 'drive', 'size']
 
for column in columns_to_change:
 print('Число уникальных значений признака {}: '.format(column), data[column].nunique())

Число уникальных значений признака cylinders:  6
Число уникальных значений признака title_status:  5
Число уникальных значений признака transmission:  3
Число уникальных значений признака drive:  3
Число уникальных значений признака size:  4


Итак, нам подходит однократное кодирование. Применим его к выбранным столбцам. Так как у нас нет отдельной тестовой выборки, то мы используем только один метод — `fit_transform()`. В качестве аргумента передаём таблицу с выбранными для преобразования признаками.

С помощью метода `get_feature_names_out()` получим список новых названий колонок:

In [7]:
from sklearn.preprocessing import OneHotEncoder
 
one_hot_encoder = OneHotEncoder()
 
# 'учим' и сразу применяем преобразование к выборке, результат переводим в массив
data_onehot = one_hot_encoder.fit_transform(data[columns_to_change]).toarray()
 
# запишем полученные названия новых колонок в отдельную переменную
column_names = one_hot_encoder.get_feature_names_out(columns_to_change)
print(column_names)

['cylinders_3' 'cylinders_4' 'cylinders_5' 'cylinders_6' 'cylinders_8'
 'cylinders_10' 'title_status_clean' 'title_status_lien'
 'title_status_missing' 'title_status_rebuilt' 'title_status_salvage'
 'transmission_automatic' 'transmission_manual' 'transmission_other'
 'drive_4wd' 'drive_fwd' 'drive_rwd' 'drive_nan' 'size_compact'
 'size_full-size' 'size_mid-size' 'size_sub-compact' 'size_nan']


Отлично, категориальные признаки закодированы. Теперь нам необходимо соединить преобразованные данные с исходными.

Напомним, что у библиотеки pandas есть дефолтный метод `get_dummies()` для получения однократного кодирования признаков. Однако `OneHotEncoder` способен принимать на вход как таблицы, так и numpy-массивы.